## Static Draft

In [ ]:
#imports
import altair as alt
import pandas as pd
import geopandas as gp
import math

In [ ]:
alt.theme.enable("fivethirtyeight")

### Load data

In [ ]:
# read in EPA NWI data from gdb file
nwi = gp.read_file("SmartLocationDatabase.gdb")

In [ ]:
for col in nwi.columns:
    print(col)

In [ ]:
# create county column from state and county fips codes
nwi["STATEFP"] = nwi["STATEFP"].astype(int)
nwi["COUNTYFP"] = nwi["COUNTYFP"].astype(int)
nwi["COUNTY5"] = nwi["STATEFP"] * 1000 + nwi["COUNTYFP"]

In [ ]:
# read in 2020 election data
# this isn't even necessary - should delete
votes_2020 = gp.read_file("2020_precincts-with-results.geojson")

In [ ]:
# get county, state, precinct from GEOID
votes_2020["GEOID"] = votes_2020["GEOID"].astype("string")
votes_2020["COUNTY5"] = votes_2020["GEOID"].str.split("-").str.get(0).astype(int)
votes_2020["STATEFP"] = votes_2020["COUNTY5"].apply(lambda x: math.floor(x/1000))
votes_2020["PRECINCT"] = votes_2020["GEOID"].str.split("-").str.get(1)

In [ ]:
# get dem lead by county
vc = votes_2020.groupby("COUNTY5", as_index=False).agg({"votes_dem": "sum", "votes_rep": "sum", "votes_total": "sum"})
vc["pct_dem_lead"] = vc["votes_dem"] / vc["votes_total"] - 0.5
votes_2020_counties = vc

In [ ]:
# merge county-level voting data to block-level walkability data
vars = ["STATEFP", "COUNTYFP", "TRACTCE", "BLKGRPCE", "COUNTY5", "CBSA",
        "TotPop", "NatWalkInd",
        "AutoOwn0", "Pct_AO0", "AutoOwn1", "Pct_AO1", "AutoOwn2p", "Pct_AO2p", 
        "Workers", "R_LowWageWk", "R_MedWageWk", "R_HiWageWk", "R_PCTLOWWAGE", "TotEmp",
        "geometry"]
nwi["CBSA"].astype(str)
nwi_2020 = pd.merge(nwi, votes_2020_counties, on="COUNTY5")

In [ ]:
# alternate: merge county-level with county-level nwi
nwi_counties = nwi.groupby(["COUNTY5", "STATEFP", "CBSA", "CBSA_Name"], as_index=False).agg({"TotPop": "sum", "NatWalkInd": "mean", "R_PCTLOWWAGE": "mean"})
nwi_2020_counties = pd.merge(nwi_counties, votes_2020_counties, on="COUNTY5")

In [ ]:
# CBSA code lookups
NY = "New York-Newark-Jersey City, NY-NJ-PA"
LA = "Los Angeles-Long Beach-Anaheim, CA"
CHI ="Chicago-Naperville-Elgin, IL-IN-WI"
HOU = "Houston-The Woodlands-Sugar Land, TX"
SEA = "Seattle-Tacoma-Bellevue, WA"
DAL = "Dallas-Fort Worth-Arlington, TX"

CBSA_lookup = {
    "35620": NY,
    "31080": LA,
    "16980": CHI,
    "26420": HOU,
    "42660": SEA,
    "19100": DAL
}
CBSA_codes = ["35620", "31080", "16980", "26420", "19100"] #"42660",

domain = [CBSA_lookup[code] for code in CBSA_codes]
range_ = ['#a463f2', '#efb118', '#ff725c', '#6cc5b0', '#4269d0']

In [ ]:
data_counties = nwi_2020_counties[nwi_2020_counties["CBSA"].isin(CBSA_codes)]
all_data = nwi_2020[nwi_2020["CBSA"].isin(CBSA_codes)]

In [ ]:
alt.data_transformers.disable_max_rows()

## Visualizations

#### Walkability by population

In [ ]:
data = all_data[["CBSA_Name", "NatWalkInd", "TotPop", "geometry"]]

In [ ]:
# TO DO: reorder by CBSA by pop descending
def nwi_by_population_by_cbsa():
    chart = alt.Chart(data).mark_bar().encode(
        alt.X("NatWalkInd:Q").bin().title("National Walkability Index"),
        alt.Y("TotPop:Q", aggregate="sum").title("Population"),
        # facet and color to visually distinguish cities
        alt.Facet("CBSA_Name:N"),
        alt.Color("CBSA_Name:N", legend=None).scale(domain=domain, range=range_),
    ).properties(
    width=300,
    height=300
    )
    return chart

nwi_by_population_by_cbsa()

In [ ]:
def nwi_pop_density(i):
    data = all_data[all_data["CBSA"] == CBSA_codes[i]]
    chart = alt.Chart(data).mark_line(color=range_[i]).encode(
        alt.X("NatWalkInd:Q"),
        alt.Y('TotPop:Q'),
    ).transform_density(
        'NatWalkInd',
        as_=['NatWalkInd', 'TotPop'],
    ).properties(title=domain[i])
    return chart


# this crashes the kernel! 
# TO DO: better way of combing density charts
"""
def nwi_pop_density_all():
    chart1 = nwi_pop_density(0)
    for i, _ in enumerate(CBSA_codes):
        chart2 = nwi_pop_density(i)
        chart1 = alt.hconcat(chart1, chart2)
    return chart1


nwi_pop_density_all()
"""

nwi_pop_density(4)


In [ ]:
def nwi_proportion_by_cbsa():
    data = all_data[["CBSA_Name", "NatWalkInd", "TotPop", "geometry"]]
    chart = alt.Chart(data).mark_bar().encode(
        alt.X("TotPop", aggregate="sum").stack("normalize").title("Proportion of Population by NWI"),
        alt.Y("CBSA_Name").title("Metropolitan Area"),
        alt.Color("NatWalkInd").bin(maxbins=10).legend(direction="horizontal", orient="top").scale(scheme="inferno").title("National Walkability Index")
    ).properties(
    width=800,
    height=300
    )
    return chart

nwi_proportion_by_cbsa()

#### Walkability vs Partisanship

In [ ]:
data = data_counties

In [ ]:
# NWI vs Dem lead (county)
chart = alt.Chart(data_counties).mark_point().encode(
    alt.X("NatWalkInd:Q").title("Walkability Index"),
    alt.Y("pct_dem_lead:Q").title("Percent Dem lead"),
    #alt.Size("TotPop:Q", legend=None),
    alt.Color("CBSA_Name:N").scale(domain=domain, range=range_),
    alt.Shape("CBSA_Name:N").scale(domain=domain)
).properties(
    width=300,
    height=300
)
chart
#chart.transform_regression('NatWalkInd', 'pct_dem_lead').mark_line()

#### Car ownership

In [ ]:
#TO DO: convert from wide-form to long-form for car ownership
# https://altair-viz.github.io/user_guide/transform/fold.html

In [ ]:
vars = ["STATEFP", "COUNTY5", "CBSA", "CBSA_Name",
        "TotPop", "NatWalkInd",
        "AutoOwn0", "Pct_AO0", "AutoOwn1", "Pct_AO1", "AutoOwn2p", "Pct_AO2p", 
        "Workers", "R_LowWageWk", "R_MedWageWk", "R_HiWageWk", "R_PCTLOWWAGE", "TotEmp",
        "geometry"]
data = all_data[vars]
auto_data = data.groupby(["CBSA", "CBSA_Name"]).agg({
    "AutoOwn0":"sum", "AutoOwn1":"sum", "AutoOwn2p":"sum",
    "Pct_AO0":"mean", "Pct_AO1":"mean", "Pct_AO2p":"mean", 
    }).reset_index()
auto_data.rename({"Pct_AO0": "0 cars", "Pct_AO1": "1 car", "Pct_AO2p": "2+ cars"}, axis='columns', inplace=True)

boroughs = [36005, 36047, 36061, 36081, 36085]
data_nyc = data[data["COUNTY5"].isin(boroughs)]
auto_data_nyc = data_nyc.groupby(["CBSA", "CBSA_Name"]).agg({
    "AutoOwn0":"sum", "AutoOwn1":"sum", "AutoOwn2p":"sum",
    "Pct_AO0":"mean", "Pct_AO1":"mean", "Pct_AO2p":"mean", 
    }).reset_index()
auto_data_nyc.rename({"Pct_AO0": "0 cars", "Pct_AO1": "1 car", "Pct_AO2p": "2+ cars"}, axis='columns', inplace=True)


In [ ]:
alt.Chart(auto_data).mark_bar().encode(
    alt.X('pct:Q').title("Percent"),
    alt.Y('auto:N', title=None),
    alt.Color('CBSA_Name').scale(domain=domain, range=range_).legend(orient='right'),
    alt.Row('CBSA')
).transform_fold(
    as_=['auto', 'pct'],
    fold=['0 cars', '1 car', '2+ cars']
)

In [ ]:
chart = alt.Chart(auto_data_nyc).mark_arc().encode(
    alt.Theta('pct:Q').title("Percent"),
    alt.Color('auto:O', title=None).scale(range=[ "#a1d99b", "#74c476", "#31a354"]), #.scale(domain=domain, range=range_),
    #alt.Color('CBSA_Name', legend=None).scale(domain=domain, range=range_),
    #alt.Row('Car ownership in New York City')
).transform_fold(
    as_=['auto', 'pct'],
    fold=['0 cars', '1 car', '2+ cars']
).properties(
    title="Car ownership in New York City"
) 
chart

In [ ]:
# transform auto ownership data for heatmap
AO_data = all_data
AO_data["NWI_bin"] = AO_data["NatWalkInd"].astype(int).round()
AO_data = AO_data.groupby("NWI_bin").agg({
    "AutoOwn0":"sum", "AutoOwn1":"sum", "AutoOwn2p":"sum",
    }).reset_index()
AO_data["total"] = AO_data["AutoOwn0"] + AO_data["AutoOwn1"] + AO_data["AutoOwn2p"]
AO_data.rename({"AutoOwn0": "0 cars", "AutoOwn1": "1 car", "AutoOwn2p": "2+ cars"}, axis='columns', inplace=True)
AO_data

In [ ]:
# heat map of NWI vs car ownership
# AI: used VS Code's built-in code completion for assistance with formatting arguments for transform_fold

alt.Chart(AO_data).mark_rect().encode(
    alt.X("NWI_bin:O", title="National Walkability Index"),
    alt.Y("auto:N", title="Auto Ownership"),
    alt.Color("TotPop:Q", title="Population").scale(scheme="greens"),
).transform_fold(
    as_=['auto', 'TotPop'],
    fold=['0 cars', '1 car', '2+ cars']
).properties(
    width=600,
    height=200,
    title="National Walkability Index vs Car Ownership"
)

In [ ]:
binned_data = all_data
binned_data["NWI_bin"] = binned_data["NatWalkInd"].astype(int).round()

In [ ]:
alt.Chart(binned_data).mark_rect().encode(
    alt.X("NWI_bin:O", title="National Walkability Index"),
    alt.Y("R_PCTLOWWAGE:N", title="Pct Low Wage Households"),
    alt.Color("TotPop:Q", title="Population").scale(type="log", scheme="greens"),
).transform_fold(
    as_=['auto', 'TotPop'],
    fold=['0 cars', '1 car', '2+ cars']
).properties(
    width=600,
    height=200,
    title="Heat map of National Walkability Index vs Car Ownership"
)

#### Wealth vs car ownership

In [ ]:
data = all_data
ny = data[data["CBSA"] == "35620"]
la = data[data["CBSA"] == "31080"]
chi = data[data["CBSA"] == "16980"]
hou = data[data["CBSA"] == "26420"]
dal = data[data["CBSA"] == "19100"]

In [ ]:
data = ny

In [ ]:
def no_car_v_wealth():

    chart1 = alt.Chart(data[data["Pct_AO0"] > 0.5]).mark_point().encode(
        alt.X("NatWalkInd:Q"),
        alt.Y("R_PCTLOWWAGE"),
        alt.Color("pct_dem_lead").scale(scheme="redblue")
    )
    chart2 = alt.Chart(data[data["Pct_AO0"] <= 0.5]).mark_point().encode(
        alt.X("NatWalkInd:Q"),
        alt.Y("R_PCTLOWWAGE"),
        alt.Color("pct_dem_lead").scale(scheme="redblue")

    )
    return chart1 | chart2

no_car_v_wealth()

#### Geo

In [ ]:
data = all_data[all_data["TotPop"] > 1]
ny = data[data["CBSA"] == "35620"].to_crs("EPSG:4326")
la = data[data["CBSA"] == "31080"].to_crs("EPSG:4326")
chi = data[data["CBSA"] == "16980"].to_crs("EPSG:4326")
hou = data[data["CBSA"] == "26420"].to_crs("EPSG:4326")
dal = data[data["CBSA"] == "19100"].to_crs("EPSG:4326")


In [ ]:
alt.Chart(ny[ny["COUNTY5"].isin(boroughs)]).mark_geoshape().encode(
    alt.Color('NatWalkInd:Q').scale(scheme='inferno')
)

In [ ]:
ny_vote = votes_2020[votes_2020["COUNTY5"].isin(ny["COUNTY5"].unique())].to_crs("EPSG:4326")
ny_vote

In [ ]:
ny_vote = votes_2020[votes_2020["COUNTY5"].isin(ny["COUNTY5"].unique())].to_crs("EPSG:4326")
alt.Chart(ny_vote).mark_geoshape().encode(
    alt.Color('pct_dem_lead:Q').scale(scheme='redblue')
)

In [ ]:
chi["COUNTY5"].unique()
chi_cook = chi[chi["COUNTY5"] == 17031]

In [ ]:
alt.Chart(chi_cook).mark_geoshape().encode(
    alt.Color('NatWalkInd:Q').scale(scheme='inferno')
).properties(
    title="Cook County Walkability",
    width=600,
    height=400
)

In [ ]:
chi_vote = votes_2020[votes_2020["COUNTY5"]==17031].to_crs("EPSG:4326")
alt.Chart(chi_vote).mark_geoshape().encode(
    alt.Color('pct_dem_lead:Q').scale(scheme='redblue').title('Percent Democratic Lead')
).properties(
    title="Cook County 2020 Vote",
    width=600,
    height=400
)

In [ ]:
alt.Chart(chi_cook).mark_geoshape().encode(
    alt.Color('D4c:Q').scale(scheme='goldgreen', reverse=True)
).properties(
    title="Transit Accessibility in Cook County",
    width=600,
    height=400
)

In [ ]:
alt.Chart(chi_cook[chi_cook["D5AR"] > 0]).mark_geoshape().encode(
    alt.Color('D5AR:Q').scale(scheme='magma', reverse=False).title("No. of jobs")
).properties(
    title="Jobs within 45 min travel time by car, Cook County",
    width=600,
    height=400
)

In [ ]:
alt.Chart(chi_cook[chi_cook["D5BR"] > 0]).mark_geoshape().encode(
    alt.Color('D5BR:Q').scale(scheme='magma', reverse=False).title("No. of jobs")
).properties(
    title="Jobs within 45 min travel time by transit, Cook County",
    width=600,
    height=400
)

#### Geopandas plots

In [ ]:
ny.plot(column="NatWalkInd", 
        cmap="YlGn", 
        legend=True, 
        figsize=(15, 10),
        legend_kwds={"label": "Walkability index", "orientation": "horizontal"})

In [ ]:
la.plot(column="NatWalkInd", 
        cmap="YlGn", 
        legend=True, 
        figsize=(15, 10),
        legend_kwds={"label": "Walkability index", "orientation": "horizontal"})

In [ ]:
chi.plot(column="NatWalkInd", 
        cmap="YlGn", 
        legend=True, 
        figsize=(15, 10),
        legend_kwds={"label": "Walkability index", "orientation": "horizontal"})

In [ ]:
hou.plot(column="NatWalkInd", 
        cmap="YlGn", 
        legend=True, 
        figsize=(15, 10),
        legend_kwds={"label": "Walkability index", "orientation": "horizontal"})

In [ ]:
dal.plot(column="NatWalkInd", 
        cmap="YlGn", 
        legend=True, 
        figsize=(15, 10),
        legend_kwds={"label": "Walkability index", "orientation": "horizontal"})